# Road Following - Live demo

In this notebook, we will use model we trained to move jetBot smoothly on track. 

### Load Trained Model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [1]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

Next, load the trained weights from the ``best_steering_model_xy.pth`` file that you uploaded.

In [2]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating the Pre-Processing Function

We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera. You should be pretty familiar with this by now. 

In [5]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()

Now, we will define sliders to control JetBot
> Note: We have initialize the slider values for best known configurations, however these might not work for your dataset, therefore please increase or decrease the sliders according to your setup and environment

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets

> Note: You should play around above mentioned sliders with lower speed to get smooth JetBot road following behavior.

Next, let's display some sliders that will let us see what JetBot is thinking.  The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value.  Please remember, this value isn't the actual angle of the target, but simply a value that is
nearly proportional.  When the actual angle is ``0``, this will be zero, and it will increase / decrease with the actual angle.  

In [7]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)

In [9]:
angle = 0.0
angle_last = 0.0
def execute(change):
    global angle, angle_last
    image = change['new']
    
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y

    steering_bias = 0.0373
    
    speed_offset = 0.10
    
    angle = np.arctan2(x, y)
    if abs(angle) < 0.05:
        angle = 0.0
        

    steering_value = angle
    steering_value *= steering_bias

    print(steering_value)
    
    robot.left_motor.value = speed_offset + steering_value/2
    robot.right_motor.value = speed_offset - steering_value/2
    
execute({'new': camera.value})
camera.observe(execute, names='value')


0.03195697547823342
0.03199721830447088
0.03199721830447088
0.03199915427406829
0.03203443229507202
0.03203443229507202
0.031871202065970994
0.031871202065970994
0.03194684134470436
0.03194684134470436
0.03199747411187513
0.03199747411187513
0.03198841518475758
0.03198841518475758
0.032093750372490876
0.032093750372490876
0.03211332238118876
0.03211332238118876
0.032023142668234486
0.032023142668234486
0.031874304288961014
0.031874304288961014
0.03183097565679351
0.03183097565679351


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [10]:
camera.observe(execute, names='value')

0.03170524648633251
0.03170524648633251
0.03171304934581024
0.03171304934581024
0.0315862225869838
0.0315862225869838
0.03163981167859863
0.03163981167859863
0.031491781913473224
0.031491781913473224


Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [11]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

0.031600536907814857
0.031600536907814857
0.03168041948794668
0.03168041948794668
0.0316742172766975
0.03161937119642998
0.03161170732420612
0.03159785714949412
0.03154131882803245
0.03166168101878074
0.03175507005406735
0.03175040189898164
0.03169903701282134
0.03195119407218932
0.031847908960104365
0.03193260762767997
0.03170063497318795
0.031739522606479637
0.031792287783351934
0.031739522606479637
0.03173331774935259
0.031705332487668714
0.031756608087855666
0.03180156145411408
0.03178455103634021
0.03188659074838246
0.03178609851998945
0.031824776959857484
0.03186938435395407
0.031762888790384525
0.031898574179975464
0.0317613139853422
0.031789175668559704
0.031878664440263285
0.03179073075708494
0.031953882988752666
0.03177989286216133
0.03163085257664633
0.031781436391762605
0.03172723488518808
0.03190166590122367
0.031866394779595776
0.03169132895624765
0.03163916265984806
0.03156930236373319
0.03185066726609798
0.031717813525126606
0.03170417443987454
0.031522528744428194
0.03

### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track follwing the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)

###### 